Members: Adam Papini, Gary Burnett, Yousuf A. Khan

The code below is for problem #1

In [ ]:
library(class)
library(MASS)

# ===== PROBLEM 1 =====
# ------ PART A ------ 

# initialize some parameters from the problem 
pi = 0.5 
K = 8 
omega = rep(1/8, K) #rep function creates K length vector of 1/8
sigma= 0.5 
sigma2 = sigma^2 

# generate location vectors 
initial_means = rbind(c(0,1), c(1,0)) # the means we will use to generate mu, 2x2 matrix of means 
mu = matrix(0, nrow = 0, ncol = 2) # initialize empty matrix 

# the first K rows of mu will be for class 0 
# the second K rows of mu will be for class 1 
for (i in 1:2){ 
  mu_x = rnorm(K, initial_means[i,1], sigma) # first component of mu for class i 
  mu_y = rnorm(K, initial_means[i,2], sigma) # second component of mu for class i 
  mu = rbind(mu, cbind(mu_x, mu_y)) 
}

# split the class means from each other 
mu_0 = mu[1:K,] # mu's (location vectors) for class 0 
mu_1 = mu[(K+1):(2*K),] # mu's (location vectors) for class 1 

# print the result 
print('location vectors for class 0 are')
print(mu_0)

print('location vectors for class 1 are')
print(mu_1)

print(sigma)
print(sigma^2)

[1] "location vectors for class 0 are"
            mu_x      mu_y
[1,]  0.57730319 0.8301554
[2,] -0.44241563 1.1932145
[3,] -0.16932508 0.8680880
[4,]  0.59194676 0.9699000
[5,]  0.13483254 1.6685943
[6,]  0.08101815 1.4380567
[7,]  0.26955509 1.0816383
[8,]  0.68652236 1.2724779
[1] "location vectors for class 1 are"
           mu_x        mu_y
[1,] 2.26121533  0.52154136
[2,] 0.90685524  0.61707022
[3,] 0.85656188  0.87328951
[4,] 0.68411953 -0.07402917
[5,] 1.20538204 -0.61736068
[6,] 0.08202829 -0.48737437
[7,] 0.89545057  0.09437323
[8,] 1.96493691 -0.74700770
[1] 0.5
[1] 0.25


In [ ]:
# ------ PART B ------ 

generate_data_points = function(y_input, centroid, N, pi, omega, sigma) {
  
  X = matrix(0, nrow = 0, ncol = 2) # initialize empty matrix to store output 

  for (i in (1:length(y_input))){

    k = floor(runif(1, min=1, max=9)) # get a random number between 0 and 8

    if (y_input[i] == 0){

      # find a random value 1:8 to choose for the gaussian
      # given the chosen K (5), choose one gaussian with one location vector (mu_5 --> sample one point from mu_5)

      single_point_0 = rnorm(1, centroid[k,1], sigma) 
      single_point_1 = rnorm(1, centroid[k,2], sigma) 

      } else {

      single_point_0 = rnorm(1, centroid[k+8,1], sigma) 
      single_point_1 = rnorm(1, centroid[k+8,2], sigma) 
      }
    
    single_point = c(single_point_0, single_point_1) 
    X = rbind(X, pi*single_point) # don't forget to multiple by pi 
  }
  
return(X) # each row is a new data point 
}


In [ ]:
# ------ PART C ------

num_train = 300 
num_test = 20000

ytrain = sample(c(0,1), replace=TRUE, size=num_train)
ytest = sample(c(0,1), replace=TRUE, size=num_test) 

xtrain = generate_data_points(ytrain, mu, num_train, 0.5, omega, sigma2) 
xtest = generate_data_points(ytest, mu, num_test, 0.5, omega, sigma2) 

In [ ]:
# ------ PART D ------


Bayes_classifier = function(means, sigma, X){
  y_pred = matrix(0, nrow = nrow(X), ncol = 1) # initialize empty matrix to store output
  K = nrow(means)/2

  class0_prior = 0.5 #the probability of our data being class 0 without seeing any features
  class1_prior = 0.5 #the probability of our data being class 1 without seeing any features

  # separate into class0 and class1
  means_class0 <- means[1:K,] 
  means_class1 <- means[(K+1):(2*K),]

  # go through formula (1)
  for (row in 1:nrow(X)){
    x = X[row,1]
    y = X[row,2]
    class0_prob = 0
    class1_prob = 0
    for (mrow in 1:K){
 
      x0_prob = dnorm(x, mean = means_class0[mrow,1], sd = sigma)
      y0_prob = dnorm(y, mean = means_class0[mrow,2], sd = sigma)

      x1_prob = dnorm(x, mean = means_class1[mrow,1], sd = sigma)
      y1_prob = dnorm(y, mean = means_class1[mrow,2], sd = sigma)

      class0_prob = class0_prob + (x0_prob*y0_prob)
      class1_prob = class1_prob + (x1_prob*y1_prob)
    }
    if (class0_prob > class1_prob){
      y_pred[row,1] = 0

    }
    else {
      y_pred[row,1] = 1
    }

  }
  # return the single column matrix of y prediction
  return(y_pred)
}



In [ ]:
# ------ PART E ------

evaluate_classifiers = function(xtrain, xtest, ytrain, ytest, k){
  
  # calculate Bayes error
  bayes_ytest_preds = Bayes_classifier(mu, sigma, xtest)
  bayes_test_error = mean((ytest-bayes_ytest_preds)^2)


  cat('Bayes test error is', bayes_test_error, '\n')
  
  # calculate least squares error
  dataset = as.data.frame(cbind(xtrain, ytrain)) # format the input data for lm function 
  ls_model = lm(ytrain~V1+V2, data=dataset) # train the ls model 
  ls_preds = predict(ls_model, as.data.frame(xtest)) # predict on the test data 
  ls_test_error = mean((ytest - ls_preds)^2) # calculate the test error 
  cat('Least Squares test error is', ls_test_error, '\n')
  
  # calculate KNN test error 
  knn_error = matrix(0, nrow=1, ncol=k)
  for (i in 1:length(k)){
    knn_pred = knn(xtrain, xtest, factor(ytrain), k=k[i]) # predict using knn 
    knn_pred = as.numeric(levels(knn_pred))[knn_pred] # convert from factor to numeric to calculate mse 
    knn_error[i] = mean((ytest - knn_pred)^2) # calculate knn test error 
    cat('KNN test error is', knn_error[i], 'on', k[i], 'clusters \n')
  }
  return(knn_error)
}

k = c(1, 3, 5, 7, 9, 11, 13, 15)
knn_error = evaluate_classifiers(xtrain, xtest, ytrain, ytest, k)

Bayes test error is 0.1506 
Least Squares test error is 0.1041706 
KNN test error is 0.17485 on 1 clusters 
KNN test error is 0.15085 on 3 clusters 
KNN test error is 0.1457 on 5 clusters 
KNN test error is 0.14015 on 7 clusters 
KNN test error is 0.1415 on 9 clusters 
KNN test error is 0.1404 on 11 clusters 
KNN test error is 0.136 on 13 clusters 
KNN test error is 0.137 on 15 clusters 


In [ ]:
# ------ PART F ------

evaluate_classifiers_modified = function(xtrain, xtest, ytrain, ytest, k, noise, sigma.noise){
  
  # add gaussian noise to xtrain and xtest
  cat('\nAdding', noise, 'columns of noise to the data with sigma.noise =', sigma.noise, '\n')
  
  num_train = dim(xtrain)[1] # number of rows in the training data 
  num_test = dim(xtest)[1] # number of rows in the test data 
  xtrain_noise = matrix(0, nrow = num_train, ncol = 0) # initialize empty matrix for training noise 
  xtest_noise = matrix(0, nrow = num_test, ncol = 0) # initialize empty matrix for testing noise 
  for (i in 1:noise){ # generate noise columns 
    xtrain_noise = cbind(xtrain_noise, rnorm(num_train, 0, sigma.noise)) 
    xtest_noise = cbind(xtest_noise, rnorm(num_test, 0, sigma.noise))
  }
  xtrain = cbind(xtrain, xtrain_noise) # append noisy columns to train data 
  xtest = cbind(xtest, xtest_noise) # append noisy columns to test data 
  
  # calculate Bayes error 
  bayes_ytest_preds = Bayes_classifier(mu, sigma, xtest)
  bayes_test_error = mean((ytest-bayes_ytest_preds)^2)
  cat('Bayes test error is', bayes_test_error, '\n')
  
  # calculate least squares error
  dataset = as.data.frame(cbind(xtrain, ytrain)) # format the input data for lm function  
  ls_model = lm(ytrain ~ ., data=dataset) # train the ls model 
  ls_preds = predict(ls_model, as.data.frame(xtest)) # predict on the test data 
  ls_test_error = mean((ytest - ls_preds)^2) # calculate the test error 
  cat('Least Squares test error is', ls_test_error, '\n')
  
  # calculate KNN test error 
  knn_noise_error = matrix(0, nrow=1, ncol=length(k)) 
  for (i in 1:length(k)){
    knn_pred = knn(xtrain, xtest, factor(ytrain), k=k[i]) # predict using knn 
    knn_pred = as.numeric(levels(knn_pred))[knn_pred] # convert from factor to numeric to calculate mse 
    knn_noise_error[i] = mean((ytest - knn_pred)^2) # calculate knn test error 
    cat('KNN test error is', knn_noise_error[i], 'on', k[i], 'clusters \n')
  }
  return(knn_noise_error)
}

In [ ]:
# ------ PART G ------

k = c(1, 3, 5, 7, 9, 11, 13, 15)
knn_noise_error = matrix(0, nrow=10, ncol=length(k))
for (n in 1:10){
  knn_noise_error[n,] = evaluate_classifiers_modified(xtrain, xtest, ytrain, ytest, k, noise=n, sigma.noise=1)
}

knn_error = rbind(knn_error, knn_noise_error)


Adding 1 columns of noise to the data with sigma.noise = 1 
Bayes test error is 0.15455 
Least Squares test error is 0.1051692 
KNN test error is 0.19125 on 1 clusters 
KNN test error is 0.1747 on 3 clusters 
KNN test error is 0.1714 on 5 clusters 
KNN test error is 0.16865 on 7 clusters 
KNN test error is 0.16655 on 9 clusters 
KNN test error is 0.16375 on 11 clusters 
KNN test error is 0.16185 on 13 clusters 
KNN test error is 0.15705 on 15 clusters 

Adding 2 columns of noise to the data with sigma.noise = 1 
Bayes test error is 0.15455 
Least Squares test error is 0.1054731 
KNN test error is 0.2263 on 1 clusters 
KNN test error is 0.20595 on 3 clusters 
KNN test error is 0.199 on 5 clusters 
KNN test error is 0.19395 on 7 clusters 
KNN test error is 0.19155 on 9 clusters 
KNN test error is 0.1875 on 11 clusters 
KNN test error is 0.1841 on 13 clusters 
KNN test error is 0.18285 on 15 clusters 

Adding 3 columns of noise to the data with sigma.noise = 1 
Bayes test error is 0.1545

Part G (Write-Up) 

As we increase the number of noise columns, we don't expect any change to the bayes test error. Bayes Classifier only relies on the prior probabilities of each class, and therefore is not influenced by noise in the training data. For least squares we expect a similar result, since the noise we are adding is relatively small and doesn't prevent the classes from being linearly separable. This is in directy contrast to KNN, which will be greatly affected by the addition of noise columns. We can see in our results that with no noise, the KNN is on par with the Bayes and Linear Classifiers, however as we increase the noise we get up to around 40% test error with 10 columns of noise. This is a significant change in performance. Another trend we notice with KNN, is that as you increase the number of neighbors, the test error decreases. This makes sense, as the classifier has more information to base its prediction off of. 

The code below is for HWK problem #3


In [ ]:
# ===== PROBLEM 3 =====
# ------ PART A ------ 
estimate_x0 = function(desired_value, N){
  x <- c(runif(N, min=0, max=1)) # get 50 random x's
  epsilon <- c(rnorm(N, 0, 0.2^2)) # get 50 random epsilon's
  f_x = (1+x^2) # our signal generator
  y <- f_x +epsilon # our signal plus noise
  d1<-data.frame(x,y) # convert into a dataframe
  model <- lm(y ~ poly(x,2), data = d1) # polynomial regression
  
  xval <- approx(x = model$fitted.values, y = x, xout = desired_value)$y
  
  return(c(xval,x,y))
}

In [ ]:
# ------ PART B ------ 
N = 50
val = 1.3
x0_single = estimate_x0(val,N)
cat('The predicted value is for a y of 1.3 is an x = ', x0_single[1])
x_original = x0_single[2:51]
y_original = x0_single[52:101]

The predicted value is for a y of 1.3 is an x =  0.5475735

In [ ]:
# ------ PART C ------
x0_1000 <- c()
for (i in 1:1000){
  x0_1000 <- c(x0_1000,(estimate_x0(val,N))[1]) #add each value
}

cat('After 1000 simulations, our predicted value is', mean(x0_1000))

After 1000 simulations, our predicted value is 0.5473172

In [ ]:
# ------ PART D ------
B = 1000  #Number times to bootstrap from our sample of 50
x0_bootstrap <- c()
for (i in 1:B){
  bootSample <- sample(1:50, 50, replace=TRUE)
  x <- c()
  y <- c()
  for (i in bootSample){
    x <- c(x,x_original[i])
    y <- c(y,y_original[i])
  }
  d1_boot <-data.frame(x,y) #convert into a dataframe
  bootModel <- lm(y ~ poly(x,2), data = d1_boot) #polynomial regression
  xval <- approx(x = bootModel$fitted.values, y = x, xout = 1.3)$y
  x0_bootstrap <- c(x0_bootstrap, xval)
}


In [ ]:
cat('The bootstrap prediction is ', mean(x0_bootstrap))

The bootstrap prediction is  0.5469935

In [ ]:
# ------ PART E ------
cat('The sampling distribution SD: ',sd(x0_1000), '\n')
cat('The bootstrap distribution SD: ', sd(x0_bootstrap))


The sampling distribution SD:  0.007470314 
The bootstrap distribution SD:  0.007663411

As you can see, the standard deviations for the sampling distribution and the bootstrap distribution are very close, as expected


# Part F

1) The first way we will do this is simply take the bottom 10% and the top 10% values as our measures of our confidence interval. The median will serve as our 'middle'.

 


In [ ]:
sorted_bootstrap <- sort(x0_bootstrap, decreasing = FALSE)
cat('The lower CI is: ', sorted_bootstrap[100], '\n')
cat('The point estimate is: ', sorted_bootstrap[500], '\n')
cat('The upper CI is: ', sorted_bootstrap[900])

The lower CI is:  0.5373687 
The point estimate is:  0.5465321 
The upper CI is:  0.5569746

2) An alternative method would be to assume that we have enough bootstrap samples to assume that the results are normally distributied therefore we could use the central limit theorm and calculate the 90% CIs from a normal value table. 


In [ ]:
z_score <- 1.645*(sd(x0_bootstrap)/sqrt(1000))
cat('The lower CI is: ', mean(x0_bootstrap) - z_score, '\n')
cat('The point estimate is: ', mean(x0_bootstrap), '\n')
cat('The upper CI is: ', mean(x0_bootstrap) + z_score)


The lower CI is:  0.5465949 
The point estimate is:  0.5469935 
The upper CI is:  0.5473922